<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/main/classify_from_image_link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image classification.  

This model architecture (resnet) has a strong history in the AI community, and it was trained by the best. However, it only knows how to label 1000 classes, mostly animals and innoccuous items. You can check the label list [here](https://huggingface.co/microsoft/resnet-50/blob/main/config.json). Such a classifier will probably only be useful to you after fine-tuning (additional training on your own data/labels)

I provided two classification models - the other model will probably be more accurate. Use resnet if you want to use a small/speedy model, want to draw upon a stronger body of literature in whatever you are publishing, or want to compare two classifiers on your data.





# Use on Your Own Data

In [11]:
#change here. 
link = "https://docs.google.com/spreadsheets/d/1UZkl8R0963yWcGxno0RZ0DBW0sEx_zZD/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"

In [12]:
# importing miscelaneaous packages 
import numpy as np # fast manipulation of multidimensional arrays
from numpy import mean
import pandas as pd

from tqdm.notebook import tqdm as progress_bar # a little vizualization of how fast a loop is running
from scipy.special import softmax
import csv
from datetime import datetime
from matplotlib.dates import date2num

# more packages, tools for getting to google drive
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import datasets

from datasets import load_dataset

In [13]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

In [15]:
from datasets import Dataset
# help us manipulate images
from PIL import Image
# helps with API request
import requests

In [16]:

import_data_from_drive(link, your_name_for_file="insta.xlsx")
df = pd.read_excel('insta.xlsx')

Using id 1UZkl8R0963yWcGxno0RZ0DBW0sEx_zZD to find file on drive
Authenticated colab user
Added file to workspace with name insta.xlsx


In [17]:
df.columns

Index(['Query Id', 'Query Name', 'Date', 'Title', 'Snippet', 'Full Text',
       'Url', 'Domain', 'Sentiment', 'Emotion', 'Page Type', 'Language',
       'Country Code', 'Continent Code', 'Continent', 'Country', 'Region Code',
       'Region', 'City Code', 'Account Type', 'Assignment', 'Author', 'Avatar',
       'Category Details', 'Checked', 'City', 'Display URLs', 'Expanded URLs',
       'Facebook Author ID', 'Facebook Comments', 'Facebook Likes',
       'Facebook Role', 'Facebook Shares', 'Facebook Subtype', 'Full Name',
       'Gender', 'Hashtags', 'Impact', 'Impressions', 'Instagram Comments',
       'Instagram Followers', 'Instagram Following', 'Instagram Likes',
       'Instagram Posts', 'Interest', 'Last Assignment Date', 'Latitude',
       'Location Name', 'Longitude', 'Media Filter', 'Media URLs',
       'Mentioned Authors', 'Priority', 'Professions', 'Resource Id',
       'Short URLs', 'Starred', 'Status', 'Subtype', 'Tags', 'Thread Author',
       'Thread Created Date', 'Th

In [18]:
import json


In [19]:
def app(url):
  return url.strip(']\'[').split(', ')
df["temporary_urls"] = df["temporary_urls"].apply(app)

In [20]:
def to_pillow(batch):
    """ a transformation to be applyed to a dataset. takes in a url string and returns data with a new column"""
    images = []
    for url_list in batch['temporary_urls']:
      # print(url_list)
      for url in url_list: # I need to get this running for multiple images per post
        # print(url)
        try:
          http_image = requests.get(url, stream=True).raw #1
          image = Image.open(http_image) # step 2
          # print(type(http_image))
          # image.show()
          # print(type(image))
        except Exception as e:
          print(e)
          image = None
        images.append(image)
    batch['images'] = images # will probably need to expand this out in practice
    return batch

In [21]:
data = Dataset.from_pandas(df)
data.set_transform(to_pillow)

In [22]:
#Alternative dataset.
# https://huggingface.co/datasets/nlphuji/flickr30k
data = load_dataset("nlphuji/flickr30k")

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
df

,Query Id,Query Name,Date,Title,Snippet,Full Text,Url,Domain,Sentiment,Emotion,...,Reach (new),Reddit Score,Reddit Score Upvote Ratio,Reddit Author Karma,Reddit Author Awardee Karma,Reddit Author Awarder Karma,Reddit Comments,Subreddit,Subreddit Subscribers,temporary_urls
0,2001569258,Sponsored IG test,2023-01-01 00:50:40.000,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,https://www.instagram.com/p/Cm2mIrYrUJb/,instagram.com,neutral,Joy,...,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-1.cdninstagram.com/v/t5...
1,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...
2,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...
3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...
4,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-1.cdninstagram.com/v/t5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,2001569258,Sponsored IG test,2022-01-02 19:47:14.000,have the holidays stressed you out? are you si...,...#sundayscaries #goth #gothgoth #sponsored #...,have the holidays stressed you out? are you si...,https://www.instagram.com/p/CYPW2pDvrOe/,instagram.com,negative,NaN,...,4081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...
1548,2001569258,Sponsored IG test,2022-01-02 11:43:01.000,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,https://www.instagram.com/p/CYOfcGLs8__/,instagram.com,neutral,NaN,...,4304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...
1549,2001569258,Sponsored IG test,2022-01-02 04:57:29.000,Come to my aid babies! 😢 I'm waiting for you i...,...#lgbt #following #toesucking #americanguys ...,Come to my aid babies! 😢 I'm waiting for you i...,https://www.instagram.com/p/CYNxB2XKj7-/,instagram.com,positive,Sadness,...,574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-1.cdninstagram.com/v/t5...
1550,2001569258,Sponsored IG test,2022-01-02 03:51:47.000,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,https://www.instagram.com/p/CYNpgrzLiSN/,instagram.com,neutral,NaN,...,2399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[https://scontent-dfw5-2.cdninstagram.com/v/t5...


In [24]:
# #Alternative dataset for testing that the model works. The datafram will not save.
# # https://huggingface.co/datasets/nlphuji/flickr30k


# data = load_dataset("nlphuji/flickr30k")

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
import math
from PIL import Image, ImageDraw
  

In [10]:
from PIL import Image
import torch
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from datasets import load_dataset


processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [27]:
predicted_labels = []
model.to('cuda')
for d in progress_bar(data['test']): # change 1

  #pass images through the preprocessor
    images = d['image']
  

  if images:
    inputs = processor(images, return_tensors="pt").to('cuda')# change 2
  else:
    predicted_labels.append(None)
    continue

  with torch.no_grad():
    logits = model(**inputs).logits

  # model predicts one of the 1000 ImageNet classes
  predicted_label = logits.argmax(-1).item()
  our_label = model.config.id2label[predicted_label]
  predicted_labels.append(our_label)





  0%|          | 0/31014 [00:00<?, ?it/s]

In [ ]:
df["predicted_label"] = predicted_labels
df.to_excel("my_data_with_class_label.xlsx")